In [2]:
import tqdm
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

# Access website using Selenium

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [9]:
chrome_path = 'C:/Users/carla/OneDrive/Área de Trabalho/chromedriver'
ser = Service(chrome_path)

In [10]:
driver = webdriver.Chrome(service=ser)

In [11]:
driver.get(url)

It was necessary to access through selenium in order to get the main table with the accidents organized by date. However, it was necessary to get the links from each accident in order to get some important information that was present in each accident page, such as the phase of flight and the classification, among others.

In [91]:
url = 'https://aviation-safety.net/database/dblist.php?Year=2011&lang=&page=1'

In [5]:
from datetime import date
current_year = date.today().year

In [92]:
years_url = []
while int(url[53:57]) <= current_year:
       years_url.append(url)
       url = url[:53]+ str(int(url[53:57])+1) +url[57:]

In [93]:
accidents = pd.DataFrame()
for year in years_url:
    table_all_pages = pd.DataFrame()
    list_url = [year]
    for url in list_url:
        driver.get(url)
        pages = [i.get_attribute('href') for i in driver.find_elements('tag name','a') if 'page'in i.get_attribute('href') ]
        pages = list(set(pages))
        [list_url.append(item) for item in pages]
        list_url = list(set(list_url))
        elem = driver.find_element('tag name','table')
        elem.get_attribute('outerHTML')
        elem.find_element('tag name','a')
        url1 = [i.get_attribute('href') for i in (elem.find_elements('tag name','a'))][1:]
        tab_page1 = pd.read_html(elem.get_attribute('outerHTML'))[0]
        tab_page1['url'] = url1
        table_all_pages = table_all_pages.append(tab_page1)
        time.sleep(5)
    accidents = accidents.append(table_all_pages)

In [95]:
accidents.to_csv('accidents-2011-2017.csv')

In [63]:
years_url = []
while int(url[53:57]) <= current_year:
       years_url.append(url)
       url = url[:53]+ str(int(url[53:57])+1) +url[57:]

In [69]:
accidents.to_csv('accidents-2018-2021.csv', index=False)

In [17]:
driver.get('https://aviation-safety.net/database/record.php?id=20201213-0')

In [36]:
 tabelaA = driver.find_elements('tag name','table')

In [79]:
tabelaA

[<selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="c76a8fec-5d12-4052-b319-8672c4d6238e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="d7ffbd4d-3d02-4779-8e86-c35ebf250765")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="a453da2e-47c2-4ae2-b0c6-13b8b0630839")>]

In [61]:
conteudo=tabelaA[0].get_attribute('textContent')
conteudo

'\nStatus:Preliminary\nDate:Sunday 13 December 2020\nTime:20:00 \nType:Eclipse 500\nOperator:private\nRegistration: N686TM\nMSN: 000221\nFirst flight: 2008 \nEngines: 2 Pratt & Whitney Canada PW610F-A\nCrew:Fatalities: 0 / Occupants: 1\nPassengers:Fatalities: 0 / Occupants: 0\nTotal:Fatalities: 0 / Occupants: 1 \n\n\n\nAircraft damage: Substantial\nLocation:Leadville Airport, CO (LXV) ( \xa0 United States of America) \n\nPhase: Landing (LDG)\nNature:Private\n\n\nDeparture airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America\n\n\nDestination airport:Leadville Airport, CO (LXV/KLXV), United States of America\n\n\n\n\n'

In [69]:
tabelaB = driver.find_elements('tag name','span')
tabelaB[1].get_attribute('textContent')

'The aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.'

In [78]:
tabelaC = driver.find_elements('xpath','//*[@id="contentcolumn"]/div/a[1]')
tabelaC[0].get_attribute('textContent')

'Runway excursion'

In [73]:
len(tabelaC)

31

Then, it was necessary to get each HTML from the links in order to create the soup, making it easier to get the information from the dataframe, using the URLs that were already found by the previous function.

In [233]:
def get_html(url):
    try:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}
        r = requests.get(url, headers=headers)
        time.sleep(7)
        return r.text
    except:
        return None

In [217]:
acc_complete['html'] = acc_complete['url'].apply(get_html)

In [219]:
acc_complete.to_csv('accidents-2011-2021-html.csv')

In [196]:
teste20 = get_html('https://aviation-safety.net/database/record.php?id=20201213-0')

In [106]:
soup = BeautifulSoup(get_html('https://aviation-safety.net/database/record.php?id=20201213-0'))

In [109]:
soup.find_all('table')[0].text

'\nStatus:Preliminary\nDate:Sunday 13 December 2020\nTime:20:00\nType:Eclipse 500\nOperator:private\nRegistration: N686TM\nMSN: 000221\nFirst flight: 2008\nEngines: 2 Pratt & Whitney Canada PW610F-A\nCrew:Fatalities: 0 / Occupants: 1\nPassengers:Fatalities: 0 / Occupants: 0\nTotal:Fatalities: 0 / Occupants: 1 \nAircraft damage: Substantial\nLocation:Leadville Airport, CO (LXV) ( \xa0 United States of America) \r\n\nPhase: Landing (LDG)\nNature:Private\nDeparture airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America\nDestination airport:Leadville Airport, CO (LXV/KLXV), United States of America\n'

In [188]:
text_list_narr =[]
for element in soup.find_all('span'):
    text_list_narr.append(element.text)
print ('\n'.join(text_list_narr))

Narrative:
The aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.


In [201]:
text_list =[]
for element in soup.find_all('table'):
    text_list.append(element.text)
    texto = '\n'.join(text_list)
print (texto)


Status:Preliminary
Date:Sunday 13 December 2020
Time:20:00
Type:Eclipse 500
Operator:private
Registration: N686TM
MSN: 000221
First flight: 2008
Engines: 2 Pratt & Whitney Canada PW610F-A
Crew:Fatalities: 0 / Occupants: 1
Passengers:Fatalities: 0 / Occupants: 0
Total:Fatalities: 0 / Occupants: 1 
Aircraft damage: Substantial
Location:Leadville Airport, CO (LXV) (   United States of America) 

Phase: Landing (LDG)
Nature:Private
Departure airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America
Destination airport:Leadville Airport, CO (LXV/KLXV), United States of America





Investigating agency: NTSB 
Status: Investigation ongoing
Accident number: CEN21LA086



Investigating agency: NTSB 
Status: Investigation ongoing
Accident number: CEN21LA086



In [20]:
import re

In [123]:
texto2 = '\n'.join(text_list)

In [136]:
teste = [re.sub(':', '', item) for item in re.findall(':.*',texto2)]
teste

['Preliminary',
 'Sunday 13 December 2020',
 '2000',
 'Eclipse 500',
 'private',
 ' N686TM',
 ' 000221',
 ' 2008',
 ' 2 Pratt & Whitney Canada PW610F-A',
 'Fatalities 0 / Occupants 1',
 'Fatalities 0 / Occupants 0',
 'Fatalities 0 / Occupants 1 ',
 ' Substantial',
 'Leadville Airport, CO (LXV) ( \xa0 United States of America) \r',
 ' Landing (LDG)',
 'Private',
 'San Diego-Montgomery Field, CA (MYF/KMYF), United States of America',
 'Leadville Airport, CO (LXV/KLXV), United States of America',
 '\xa0NTSB ',
 ' Investigation ongoing',
 ' CEN21LA086',
 '\xa0NTSB ',
 ' Investigation ongoing',
 ' CEN21LA086']

In [195]:
teste = [re.sub(':\xa0', '', item) for item in re.findall(':.*',text_list[2])]
teste[0]

'NTSB '

# Getting information from the HTML column

In [282]:
def details_phase(html):
    try:
        soup = BeautifulSoup(html)
        text_list3 =[]
        for element in soup.find_all('table'):
            text_list3.append(element.text)
        if len(re.findall('Phase((.|\n)*?)(Nature)',text_list3[0]))>0:
            return re.findall('Phase((.|\n)*?)(Nature)',text_list3[0])[0][0]
        else:
            return pd.NA
    except:
        return pd.NA

In [283]:
def details_damage(html):
    try:
        soup = BeautifulSoup(html)
        text_list2 = []
        for element in soup.find_all('table'):
            text_list2.append(element.text)
        if len(re.findall('damage((.|\n)*?)(Aircraft|Location|Phase)',text_list2[0]))>0:
            return re.findall('damage((.|\n)*?)(Aircraft|Location|Phase)',text_list2[0])[0][0]
        else:
            return pd.NA
    except:
        return pd.NA

In [258]:
def details_nature(html):
    try:
        soup = BeautifulSoup(html)
        text_list4 =[]
        for element in soup.find_all('table'):
            text_list4.append(element.text)
        cleaning = [re.sub(':', '', item) for item in re.findall(':.*',text_list4[0])]
        if len(re.findall('Nature((.|\n)*?)(Departure)',text_list4[0]))>0:
            return re.findall('Nature((.|\n)*?)(Departure)',text_list4[0])[0][0]
        else:
            return None
    except:
        return None

In [206]:
details1(teste20, 14)

' Landing (LDG)'

In [284]:
def details_inv(html):
    try:
        soup = BeautifulSoup(html)
        text_list =[]
        for element in soup.find_all('table'):
            text_list.append(element.text)
        cleaning = [re.sub(':\xa0', '', item) for item in re.findall(':.*',text_list[2])]
        return cleaning[0]
    except:
        return pd.NA


In [210]:
details2(teste20)

'NTSB '

In [285]:
def details3(html):
    try:
        soup = BeautifulSoup(html)
        text_list_narr =[]
        for element in soup.find_all('span'):
            text_list_narr.append(element.text)
        return ('\n'.join(text_list_narr))
    except:
        return pd.NA

In [212]:
details3(teste20)

'Narrative:\nThe aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.'

In [286]:
def details_class(html):
    try:
        soup = BeautifulSoup(html)
        text_list2 = []
        for element in soup.find_all('div', attrs={'class':'innertube'}):
            text_list2.append(element.text)
        if len(re.findall('Classification((.|\n)*?)(Source|Photos|Follow-up|Weather|METAR)',text_list2[0]))>0:
            return re.findall('Classification((.|\n)*?)(Source|Photos|Follow-up|Weather|METAR)',text_list2[0])[0][0]
        else:
            return pd.NA
    except:
        return pd.NA

In [273]:
pd.NA

<NA>

In [254]:
acc_complete = pd.read_csv('accidents-2011-2021-html.csv')

In [235]:
url_null = acc_complete[acc_complete['html'].isnull()]['url'].apply(get_html)


In [269]:
acc_complete.loc[acc_complete['html'].isnull(),'html']=url_null

In [274]:
acc_complete.to_csv('accidents-html-complete')

In [287]:
acc_complete['phase'] = acc_complete['html'].apply(details_phase)

In [288]:
acc_complete['aircraft_damage'] = acc_complete['html'].apply(details_damage)

In [289]:
acc_complete['classification'] = acc_complete['html'].apply(details_class)

In [290]:
acc_complete['narrative'] = acc_complete['html'].apply(details3)

In [291]:
acc_complete['nature'] = acc_complete['html'].apply(details_nature)

In [292]:
acc_complete['inv_agency'] = acc_complete['html'].apply(details_inv)

In [249]:
#html_acc = pd.read_csv('accidents-2011-2021-html.csv')

# Cleaning the content

In [296]:
def cleaning(string):
    text = (re.sub('[:\n]', '', (str(string)))).strip()
    return text


In [307]:
def cleaning_narr(string):
    text = (re.sub('Narrative', '', (str(string)))).strip()
    return text

In [297]:
def cleaning_class(string):
    text = (re.sub('[:\n]', ' ', (str(string)))).strip()
    return text

In [298]:
acc_complete['aircraft_damage'] = acc_complete['aircraft_damage'].apply(cleaning)

In [299]:
acc_complete['phase'] = acc_complete['phase'].apply(cleaning)

In [300]:
acc_complete['nature'] = acc_complete['nature'].apply(cleaning)

In [301]:
acc_complete['narrative'] = acc_complete['narrative'].apply(cleaning)

In [308]:
acc_complete['narrative'] = acc_complete['narrative'].apply(cleaning_narr)

In [302]:
acc_complete['classification'] = acc_complete['classification'].apply(cleaning_class)

In [306]:
acc_complete = acc_complete.drop(columns=['Unnamed: 0','Unnamed: 6', 'pic', 'cat'])

In [311]:
accidents_complete = acc_complete.drop(columns=['html'])
accidents_complete

,date,type,registration,operator,fat.,location,url,phase,aircraft_damage,classification,narrative,nature,inv_agency
0,01-JAN-2011,Tupolev Tu-154B-2,RA-85588,Kolavia,3,Surgut Airpo...,https://aviation-safety.net/database/record.ph...,Standing (STD),Destroyed,Damaged on the ground,"A Tupolev 154B-2 passenger jet, registered RA-...",Domestic Scheduled Passenger,MAK
1,03-JAN-2011,Beech B200 King Air,C-GSAU,Saskatchewan Gvt.,0,Maple Creek ...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Substantial,Runway excursion,"A Beechcraft B200 King Air, C-GSAU, sustained ...",Ambulance,TSB Canada
2,03-JAN-2011,Boeing 737-823 (WL),N831NN,American Airlines,0,Los Angeles ...,https://aviation-safety.net/database/record.ph...,Takeoff (TOF),Substantial,Tailstrike Runway mishap,"American Airlines flight 1586, a Boeing 737-80...",International Scheduled Passenger,NTSB
3,05-JAN-2011,Boeing 737-8F2 (WL),TC-JGZ,THY,0,Istanbul Ata...,https://aviation-safety.net/database/record.ph...,En route (ENR),None,Hijack,A 40 year old man attempted hijack Flight 1754...,International Scheduled Passenger,<NA>
4,06-JAN-2011,Learjet 35A,N800GP,Priester Aviation,0,Springfield-...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Damaged beyond repair,Icing Loss of control,"A Learjet 35A corporate jet, registered N800GP...",Executive,NTSB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,13-NOV-2021,BN-2A-6 Islander,N866JA,Island Airways,4,near Beaver Islan...,https://aviation-safety.net/database/record.ph...,Approach (APR),Destroyed,<NA>,"A Britten-Norman BN-2A Islander, was destroyed...",Domestic Non Scheduled Passenger,NTSB
2188,14-NOV-2021,IAI 1125 Astra SP,N771DX,EMD Astra Holdings LLC,0,Las Vegas-No...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Substantial,Runway excursion,"An IAI 1125 Astra SP jet, N771DX, sustained su...",Private,<NA>
2189,29-NOV-2021,Boeing 747-4B5F,N705CK,Kalitta Air,0,between Sant...,https://aviation-safety.net/database/record.ph...,En route (ENR),Substantial,<NA>,"Kalitta Air flight K4538A, a Boeing 747-4B5F, ...",Cargo,NTSB
2190,01-DEC-2021,Cessna 208 Caravan I,N9602F,Lake & Peninsula Airlines,0,near Port Alswort...,https://aviation-safety.net/database/record.ph...,Approach (APR),Substantial,<NA>,The Cessna 208 Caravan I crashed into trees on...,Unknown,<NA>


In [315]:
accidents_complete.to_csv('accidents-2011-2021-cleaned.csv',index=False)

# Getting global info (total of flight and passengers)

In [227]:
passengers = {'year':[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], 
'total_passengers':[2859000000, 2991000000, 3145000000, 3328000000, 3569000000, 3817000000, 4095000000, 4378000000, 4543000000, 4723000000, 2277000000], 
'total_flights':[38000000, ]}

In [316]:
pd.read_csv('accidents-2011-2021-cleaned.csv')

,date,type,registration,operator,fat.,location,url,phase,aircraft_damage,classification,narrative,nature,inv_agency
0,01-JAN-2011,Tupolev Tu-154B-2,RA-85588,Kolavia,3,Surgut Airpo...,https://aviation-safety.net/database/record.ph...,Standing (STD),Destroyed,Damaged on the ground,"A Tupolev 154B-2 passenger jet, registered RA-...",Domestic Scheduled Passenger,MAK
1,03-JAN-2011,Beech B200 King Air,C-GSAU,Saskatchewan Gvt.,0,Maple Creek ...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Substantial,Runway excursion,"A Beechcraft B200 King Air, C-GSAU, sustained ...",Ambulance,TSB Canada
2,03-JAN-2011,Boeing 737-823 (WL),N831NN,American Airlines,0,Los Angeles ...,https://aviation-safety.net/database/record.ph...,Takeoff (TOF),Substantial,Tailstrike Runway mishap,"American Airlines flight 1586, a Boeing 737-80...",International Scheduled Passenger,NTSB
3,05-JAN-2011,Boeing 737-8F2 (WL),TC-JGZ,THY,0,Istanbul Ata...,https://aviation-safety.net/database/record.ph...,En route (ENR),None,Hijack,A 40 year old man attempted hijack Flight 1754...,International Scheduled Passenger,NaN
4,06-JAN-2011,Learjet 35A,N800GP,Priester Aviation,0,Springfield-...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Damaged beyond repair,Icing Loss of control,"A Learjet 35A corporate jet, registered N800GP...",Executive,NTSB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,13-NOV-2021,BN-2A-6 Islander,N866JA,Island Airways,4,near Beaver Islan...,https://aviation-safety.net/database/record.ph...,Approach (APR),Destroyed,NaN,"A Britten-Norman BN-2A Islander, was destroyed...",Domestic Non Scheduled Passenger,NTSB
2188,14-NOV-2021,IAI 1125 Astra SP,N771DX,EMD Astra Holdings LLC,0,Las Vegas-No...,https://aviation-safety.net/database/record.ph...,Landing (LDG),Substantial,Runway excursion,"An IAI 1125 Astra SP jet, N771DX, sustained su...",Private,NaN
2189,29-NOV-2021,Boeing 747-4B5F,N705CK,Kalitta Air,0,between Sant...,https://aviation-safety.net/database/record.ph...,En route (ENR),Substantial,NaN,"Kalitta Air flight K4538A, a Boeing 747-4B5F, ...",Cargo,NTSB
2190,01-DEC-2021,Cessna 208 Caravan I,N9602F,Lake & Peninsula Airlines,0,near Port Alswort...,https://aviation-safety.net/database/record.ph...,Approach (APR),Substantial,NaN,The Cessna 208 Caravan I crashed into trees on...,Unknown,NaN
